Lab | Making predictions with logistic regression

In this lab, you will be using the Sakila database of movie rentals.

In order to optimize our inventory, we would like to know which films will be rented. We are asked to create a model to predict it. So we use the information we have from May 2005 to create the model.

Instructions

1. Create a query or queries to extract the information you think may be relevant for building the prediction model. It should include some film features and some rental features (X).

2. Create a query to get the list of all unique film titles and a boolean indicating if it was rented (rental_date) in May 2005. (Create new column called - 'rented_in_may'). This will be our TARGET (y) variable.

3. Read the data into a Pandas dataframe. At this point you should have 1000 rows. Number of columns depends on the number of features you chose.

4. Analyze extracted features (X) and transform them. You may need to encode some categorical variables, or scale numerical variables.

5. Create a logistic regression model to predict 'rented_in_may' from the cleaned data.

6. Evaluate the results.

In [1]:
import pymysql
from sqlalchemy import create_engine
import pandas as pd
import numpy as np
import getpass
password = getpass.getpass()

········


In [2]:
connection_string = 'mysql+pymysql://root:' + password + '@localhost/sakila'

In [3]:
engine = create_engine(connection_string)

In [4]:
# Create a query or queries to extract the information you think may be relevant for building the prediction model. 
# It should include some film features and some rental features (X):

In [5]:
query1 = '''SELECT f.title, f.rental_rate, f.length, f.rating, r.rental_date, p.amount, c.name
FROM film f
LEFT JOIN inventory i
USING (film_id)
LEFT JOIN rental r
USING (inventory_id)
LEFT JOIN payment p
USING (rental_id)
LEFT JOIN film_category fc
USING (film_id)
LEFT JOIN category c
USING (category_id)
;'''

In [6]:
data = pd.read_sql_query(query1, engine)
display(data.head())
data.shape

,title,rental_rate,length,rating,rental_date,amount,name
0,ACADEMY DINOSAUR,0.99,86,PG,2005-07-08 19:03:15,0.99,Documentary
1,ACADEMY DINOSAUR,0.99,86,PG,2005-08-02 20:13:10,3.99,Documentary
2,ACADEMY DINOSAUR,0.99,86,PG,2005-08-21 21:27:43,3.99,Documentary
3,ACADEMY DINOSAUR,0.99,86,PG,2005-05-30 20:21:07,1.99,Documentary
4,ACADEMY DINOSAUR,0.99,86,PG,2005-06-17 20:24:00,0.99,Documentary


(16087, 7)

In [7]:
# Create a query to get the list of all unique film titles and a boolean indicating if it was rented (rental_date) 
# in May 2005. (Create new column called - 'rented_in_may'). This will be our TARGET (y) variable:

In [8]:
query2='''SELECT
    f.title,
    f.rental_rate,
    f.length,
    f.rating,
    MAX(IF(MONTH(r.rental_date) = 5 AND YEAR(r.rental_date) = 2005, 1, 0)) AS rented_in_may,
    AVG(p.amount) AS amount,
    MAX(c.name) AS name
FROM film f
LEFT JOIN inventory i USING (film_id)
LEFT JOIN rental r USING (inventory_id)
LEFT JOIN payment p USING (rental_id)
LEFT JOIN film_category fc USING (film_id)
LEFT JOIN category c USING (category_id)
GROUP BY f.title, f.rental_rate, f.length, f.rating;'''


In [9]:
data = pd.read_sql_query(query2, engine)
display(data.head())
data.shape

,title,rental_rate,length,rating,rented_in_may,amount,name
0,ACADEMY DINOSAUR,0.99,86,PG,1,1.598696,Documentary
1,ACE GOLDFINGER,4.99,48,G,0,7.561429,Horror
2,ADAPTATION HOLES,2.99,50,NC-17,1,3.156667,Documentary
3,AFFAIR PREJUDICE,2.99,117,G,1,3.990000,Horror
4,AFRICAN EGG,2.99,130,G,1,4.323333,Family


(1000, 7)

In [10]:
data.isna().sum()

title             0
rental_rate       0
length            0
rating            0
rented_in_may     0
amount           42
name              0
dtype: int64

In [11]:
data['unrented'] = data['amount'].isna().astype(int)

In [12]:
data.head()

,title,rental_rate,length,rating,rented_in_may,amount,name,unrented
0,ACADEMY DINOSAUR,0.99,86,PG,1,1.598696,Documentary,0
1,ACE GOLDFINGER,4.99,48,G,0,7.561429,Horror,0
2,ADAPTATION HOLES,2.99,50,NC-17,1,3.156667,Documentary,0
3,AFFAIR PREJUDICE,2.99,117,G,1,3.990000,Horror,0
4,AFRICAN EGG,2.99,130,G,1,4.323333,Family,0


In [13]:
data.unrented.value_counts()

0    958
1     42
Name: unrented, dtype: int64

In [14]:
data.isna().sum()

title             0
rental_rate       0
length            0
rating            0
rented_in_may     0
amount           42
name              0
unrented          0
dtype: int64

In [15]:
data['amount'] = data['amount'].fillna(data['amount'].median())
data['amount'] = data['amount'].round(2)

In [16]:
data.head()

,title,rental_rate,length,rating,rented_in_may,amount,name,unrented
0,ACADEMY DINOSAUR,0.99,86,PG,1,1.60,Documentary,0
1,ACE GOLDFINGER,4.99,48,G,0,7.56,Horror,0
2,ADAPTATION HOLES,2.99,50,NC-17,1,3.16,Documentary,0
3,AFFAIR PREJUDICE,2.99,117,G,1,3.99,Horror,0
4,AFRICAN EGG,2.99,130,G,1,4.32,Family,0


In [17]:
data.isna().sum()

title            0
rental_rate      0
length           0
rating           0
rented_in_may    0
amount           0
name             0
unrented         0
dtype: int64

In [18]:
data.dtypes

title             object
rental_rate      float64
length             int64
rating            object
rented_in_may      int64
amount           float64
name              object
unrented           int64
dtype: object

In [19]:
data.rating.value_counts()

PG-13    223
NC-17    210
R        195
PG       194
G        178
Name: rating, dtype: int64

In [20]:
X_num = data.select_dtypes(include = np.number)
X_cat = data.select_dtypes(include = object)

display(X_num)
display(X_cat)

,rental_rate,length,rented_in_may,amount,unrented
0,0.99,86,1,1.60,0
1,4.99,48,0,7.56,0
2,2.99,50,1,3.16,0
3,2.99,117,1,3.99,0
4,2.99,130,1,4.32,0
...,...,...,...,...,...
995,0.99,183,0,0.99,0
996,0.99,179,0,2.82,0
997,0.99,105,1,1.66,0
998,2.99,101,1,4.34,0


,title,rating,name
0,ACADEMY DINOSAUR,PG,Documentary
1,ACE GOLDFINGER,G,Horror
2,ADAPTATION HOLES,NC-17,Documentary
3,AFFAIR PREJUDICE,G,Horror
4,AFRICAN EGG,G,Family
...,...,...,...
995,YOUNG LANGUAGE,G,Documentary
996,YOUTH KICK,NC-17,Music
997,ZHIVAGO CORE,NC-17,Horror
998,ZOOLANDER FICTION,R,Children


In [21]:
y = data["rented_in_may"]
X = data.drop("rented_in_may", axis=1)
display(X)

,title,rental_rate,length,rating,amount,name,unrented
0,ACADEMY DINOSAUR,0.99,86,PG,1.60,Documentary,0
1,ACE GOLDFINGER,4.99,48,G,7.56,Horror,0
2,ADAPTATION HOLES,2.99,50,NC-17,3.16,Documentary,0
3,AFFAIR PREJUDICE,2.99,117,G,3.99,Horror,0
4,AFRICAN EGG,2.99,130,G,4.32,Family,0
...,...,...,...,...,...,...,...
995,YOUNG LANGUAGE,0.99,183,G,0.99,Documentary,0
996,YOUTH KICK,0.99,179,NC-17,2.82,Music,0
997,ZHIVAGO CORE,0.99,105,NC-17,1.66,Horror,0
998,ZOOLANDER FICTION,2.99,101,R,4.34,Children,0


In [22]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [23]:
X_train_num = X_train.select_dtypes(include = np.number)

display(X_train_num)

,rental_rate,length,amount,unrented
29,2.99,82,5.21,0
535,2.99,65,4.70,0
695,0.99,114,1.18,0
557,2.99,71,4.79,0
836,2.99,85,4.69,0
...,...,...,...,...
106,2.99,63,5.19,0
270,4.99,148,6.56,0
860,4.99,111,7.04,0
435,0.99,122,2.62,0


In [24]:
from sklearn.preprocessing import MinMaxScaler

transformer = MinMaxScaler().fit(X_train_num) 
X_train_normalized = transformer.transform(X_train_num)
X_train_norm = pd.DataFrame(X_train_normalized, columns=X_train_num.columns)
X_train_norm

,rental_rate,length,amount,unrented
0,0.5,0.258993,0.549479,0.0
1,0.5,0.136691,0.483073,0.0
2,0.0,0.489209,0.024740,0.0
3,0.5,0.179856,0.494792,0.0
4,0.5,0.280576,0.481771,0.0
...,...,...,...,...
795,0.5,0.122302,0.546875,0.0
796,1.0,0.733813,0.725260,0.0
797,1.0,0.467626,0.787760,0.0
798,0.0,0.546763,0.212240,0.0


In [25]:
X_train_cat = X_train.select_dtypes(include = object)

X_train_cat

,title,rating,name
29,ANYTHING SAVANNAH,R,Horror
535,LOVELY JINGLE,PG,Classics
695,PRIDE ALAMO,NC-17,Animation
557,MANNEQUIN WORST,PG-13,New
836,STAGE WORLD,PG,Comedy
...,...,...,...
106,BUNCH MINDS,G,Drama
270,EASY GLADIATOR,G,Action
860,SUIT WALLS,R,Games
435,HOURS RAGE,NC-17,New


In [26]:
from sklearn.preprocessing import OneHotEncoder, LabelEncoder

label_encoder = LabelEncoder()
one_hot_encoder = OneHotEncoder(drop='first', sparse_output=False)

for col in X_train_cat.columns:
    X_train_cat[col] = label_encoder.fit_transform(X_train_cat[col])

onehot_encoded = one_hot_encoder.fit_transform(X_train_cat)
column_names = one_hot_encoder.get_feature_names_out(input_features=X_train_cat.columns)
onehot_encoded = pd.DataFrame(onehot_encoded, columns=column_names)

display(onehot_encoded.head())
onehot_encoded.shape


,title_1,title_2,title_3,title_4,title_5,title_6,title_7,title_8,title_9,title_10,...,name_6,name_7,name_8,name_9,name_10,name_11,name_12,name_13,name_14,name_15
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


(800, 818)

In [27]:
X_train_transformed = np.concatenate([X_train_norm, X_train_cat], axis=1)

X_train_transformed

array([[5.00000000e-01, 2.58992806e-01, 5.49479167e-01, ...,
        2.60000000e+01, 4.00000000e+00, 1.00000000e+01],
       [5.00000000e-01, 1.36690647e-01, 4.83072917e-01, ...,
        4.31000000e+02, 2.00000000e+00, 3.00000000e+00],
       [0.00000000e+00, 4.89208633e-01, 2.47395833e-02, ...,
        5.53000000e+02, 1.00000000e+00, 1.00000000e+00],
       ...,
       [1.00000000e+00, 4.67625899e-01, 7.87760417e-01, ...,
        6.90000000e+02, 4.00000000e+00, 9.00000000e+00],
       [0.00000000e+00, 5.46762590e-01, 2.12239583e-01, ...,
        3.49000000e+02, 1.00000000e+00, 1.20000000e+01],
       [1.00000000e+00, 6.25899281e-01, 5.62500000e-01, ...,
        8.10000000e+01, 2.00000000e+00, 1.50000000e+01]])

In [28]:
from sklearn.linear_model import LogisticRegression

classification = LogisticRegression(random_state=0, solver='newton-cg', max_iter=1000, multi_class='multinomial').fit(X_train_transformed, y_train)

In [29]:
X_test_num = X_test.select_dtypes(include = np.number)

X_test_normalized = transformer.transform(X_test_num)
X_test_norm = pd.DataFrame(X_test_normalized, columns=X_test_num.columns)
X_test_norm

,rental_rate,length,amount,unrented
0,0.5,0.654676,0.455729,0.0
1,0.0,0.949640,0.332031,0.0
2,0.0,0.251799,0.032552,0.0
3,1.0,0.798561,0.601563,0.0
4,1.0,0.402878,0.605469,0.0
...,...,...,...,...
195,1.0,0.093525,0.735677,0.0
196,0.5,0.575540,0.276042,0.0
197,0.5,0.381295,0.360677,0.0
198,1.0,0.949640,0.645833,0.0


In [30]:
X_test_cat = X_test.select_dtypes(include='object')
    
label_encoder = LabelEncoder()
one_hot_encoder = OneHotEncoder(drop='first', sparse_output=False)

for col in X_test_cat.columns:
    X_test_cat[col] = label_encoder.fit_transform(X_test_cat[col])

onehot_encoded = one_hot_encoder.fit_transform(X_test_cat)
column_names = one_hot_encoder.get_feature_names_out(input_features=X_test_cat.columns)
onehot_encoded = pd.DataFrame(onehot_encoded, columns=column_names)

onehot_encoded

,title_1,title_2,title_3,title_4,title_5,title_6,title_7,title_8,title_9,title_10,...,name_6,name_7,name_8,name_9,name_10,name_11,name_12,name_13,name_14,name_15
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
196,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
197,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
198,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [31]:
list(X_train_cat.columns)==list(X_test_cat.columns)

True

In [32]:
X_test_transformed = np.concatenate([X_test_norm, X_test_cat], axis=1)

In [33]:
predictions = classification.predict(X_test_transformed)
display(predictions)
classification.score(X_test_transformed, y_test)

array([1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1])

0.72

In [34]:
print(y_test.value_counts())

1    135
0     65
Name: rented_in_may, dtype: int64


In [35]:
pd.Series(predictions).value_counts()

1    191
0      9
dtype: int64

In [36]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, predictions)

array([[  9,  56],
       [  0, 135]])

In [37]:
predictions_df = pd.DataFrame(predictions, columns=["Predicted_Rented_in_May"])


In [38]:
test_with_predictions = pd.concat([X_test, predictions_df], axis=1)
test_with_predictions

,title,rental_rate,length,rating,amount,name,unrented,Predicted_Rented_in_May
521,LIFE TWISTED,2.99,137.0,NC-17,4.49,Comedy,0.0,NaN
737,ROCKETEER MOTHER,0.99,178.0,PG-13,3.54,Foreign,0.0,NaN
740,ROMAN PUNK,0.99,81.0,NC-17,1.24,Music,0.0,NaN
660,PAST SUICIDES,4.99,157.0,PG-13,5.61,Foreign,0.0,NaN
411,HEAVYWEIGHTS BEAST,4.99,102.0,G,5.64,Documentary,0.0,NaN
...,...,...,...,...,...,...,...,...
193,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
194,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
195,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
196,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0


In [39]:
test_with_predictions = test_with_predictions.dropna()

In [40]:
test_with_predictions

,title,rental_rate,length,rating,amount,name,unrented,Predicted_Rented_in_May
136,CHARADE DUFFEL,2.99,66.0,PG,5.39,Sci-Fi,0.0,1.0
76,BIRDS PERDITION,4.99,61.0,G,6.21,New,0.0,1.0
174,CONFUSED CANDLES,2.99,122.0,PG-13,6.05,Family,0.0,1.0
101,BUBBLE GROSSE,4.99,60.0,R,5.99,Sports,0.0,1.0
139,CHEAPER CLYDE,0.99,87.0,G,1.84,Sci-Fi,0.0,1.0
198,CUPBOARD SINNERS,2.99,56.0,R,4.64,Documentary,0.0,1.0
59,BEAST HUNCHBACK,4.99,89.0,R,6.99,Classics,0.0,1.0
96,BRIDE INTRIGUE,0.99,56.0,G,1.15,Action,0.0,1.0
23,ANALYZE HOOSIERS,2.99,181.0,R,3.99,Horror,0.0,1.0
30,APACHE DIVINE,4.99,92.0,NC-17,5.76,Family,0.0,1.0
